In [83]:
from pathlib import Path
from shutil import move
from pandas import Timestamp, read_sql
from tqdm import tqdm
from model import engine
from humanize import naturalsize


In [84]:
! python -u refresh_db.py

Stage: REFRESH DB

Loading current database: 63443 rows
Creating dirs dataframe : 63442 rows

New rows: 0 | Delete rows: 1
Updating db
Committing changes
Stage complete



Adding new rows: 0files [00:00, ?files/s]
Adding new rows: 0files [00:00, ?files/s]

Remove old rows: 100%|██████████| 1/1 [00:00<?, ?files/s]


In [85]:
def tag(row):
    dt = row["exif_date"] if type(row["exif_date"]) == Timestamp else row["modify_date"]
    return str(dt)[:7]

df = read_sql("google_photos", con=engine)
df["path"] = df["path"].apply(Path) #type: ignore
df["tag"] = df.apply(tag, axis=1)
df.sort_values(by="tag", inplace=True)
print(df.shape)

(63442, 12)


In [86]:
tag = df.iloc[0]["tag"]
print(f"{tag=}")
part = df[df["tag"] == tag]
print(f"Total: {len(part)} | Size: {naturalsize(sum(part["size"]))}")
# print(part["filename"].value_counts())

tag='2018-01'
Total: 201 | Size: 338.2 MB


In [87]:
if (part["filename"].value_counts().iloc[0] == 1):
    root = Path(f"upload/{tag}")
    root.mkdir(parents=True, exist_ok=True)
    list(map(lambda file: move(file, root/file.name), tqdm(part["path"])))
else:
    print("Duplicates found, handle themn.")

100%|██████████| 201/201 [00:00<00:00, 879.62it/s] 


In [88]:
part["filename"].value_counts()

filename
B612_20180116_162206.jpg          1
B612_20180112_140409.jpg          1
IMG_20180116_165105.jpg           1
IMG_20180115_182357.jpg           1
IMG_20180116_170056.jpg           1
                                 ..
IMG_20180115_104712.jpg           1
IMG_20180121_174837.jpg           1
IMG_20180116_164921.jpg           1
IMG_20180116_163425_BURST1.jpg    1
IMG_20180115_120125.jpg           1
Name: count, Length: 201, dtype: int64

In [ ]:
part[part["filename"] == "IMG_20170926_154020.jpg"]

,filename,path,type,size,size_norm,media,modify_date,exif_date,file_hash,media_hash,exif_data,tag


: 